In [1]:
#Things to Do
#11.1 Create a file called zoo.py. In it, define a function called hours() that prints the string 'Open 9-5 daily'. Then, use the interactive interpreter
#to import the zoo module and call its hours() function.

import zoo
zoo.hours()


Open 9-5 daily


In [2]:
# 11.2 In the interactive interpreter, import the zoo module as menagerie and call its hours() function.
import zoo as menagerie
menagerie.hours()

Open 9-5 daily


In [ ]:
# 1. IMPORTING NECESSARY MODULES
# Flask: The core web framework.
# request: To handle incoming data (like JSON from a POST request).
# jsonify: To turn Python dictionaries into JSON responses for the user.
# SQLAlchemy: The ORM (Object Relational Mapper) that lets us use Python instead of raw SQL.

from flask import Flask, request, jsonify
from flask_sqlalchemy import SQLAlchemy

# 2. APP INITIALIZATION & CONFIGURATION
app = Flask(__name__)

# Set the database location. 'sqlite:///books.db' creates a file in your project folder.
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///books.db'

# Disable tracking overhead to improve performance.
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False

# Link the SQLAlchemy instance to our Flask app.
db = SQLAlchemy(app)

# 3. DEFINING THE DATA MODEL:This class defines what a "Book" looks like in our database tables.
# BOOK MODEL
class Book(db.Model):
    id = db.Column(db.Integer, primary_key=True) # Unique ID for every book
    book_name = db.Column(db.String(100), nullable=False)
    author = db.Column(db.String(100), nullable=False)
    publisher = db.Column(db.String(100), nullable=False) # type: ignore

    # Helper method to convert the database object into a dictionary.Essential for returning JSON data.
    def to_dict(self):
        return {
            "id": self.id,
            "book_name": self.book_name,
            "author": self.author,
            "publisher": self.publisher
        }

# 4. DATABASE CREATION: with app.app_context() ensures the app is active when we trigger database creation.
# Create DB

with app.app_context():
    db.create_all()

# 5. CREATE: ADD A NEW BOOK

@app.route('/books', methods=['POST'])
def add_book():
    data = request.get_json() # Extract JSON data from the request body
    new_book = Book(
       book_name=data['book_name'],
       author=data['author'],
       publisher=data['publisher']
       )
    db.session.add(new_book) #Add the book to the staging area
    db.session.commit()      #Save changes to the database
    return jsonify(new_book.to_dict()), 201   #Return the new book and a 201 "Created" Status

# 6. READ ALL: FETCH ALL BOOKS

@app.route('/books', methods=['GET'])
def get_all_books():
    books = Book.query.all()
    return jsonify([book.to_dict() for book in books])

# 7. READ ONE: FETCH BY ID
@app.route('/books/<int:id>' , methods=['GET'])
def get_one_book(id):
    book = Book.query.get_or_404(id)
    return jsonify(book.to_dict())
    

# 8. UPDATE: MODIFY AN EXISTING BOOK
@app.route('/books/<int:id>' , methods=['PUT'])
def update_book(id):
    book = Book.query.get_or_404(id)
    data = request.get_json()

    # .get() allows us to update only the fields provided, keeping old values as defaults. 

    book.book_name = data.get('book_name', book.book_name) 
    book.author    = data.get('author', book.author)        
    book.publisher = data.get('publisher', book.publisher)  
    # Save the updates to the database
    db.session.commit()  
    return jsonify({"message": "Book updated successfully"})
 


# 9. DELETE: REMOVE A BOOK
@app.route('/books/<int:id>', methods=['DELETE'])
def delete_book(id):
    book = Book.query.get_or_404(id)
    db.session.delete(book)  # Mark the record for deletion
    db.session.commit()      # Execute the deletion
    return jsonify({"message": "Book deleted"})

# 10. RUN THE SERVER
if __name__ == '__main__':
    # use_reloader=False stops the server from crashing in Jupyter
    app.run(debug=True, use_reloader=False)

   

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [07/Feb/2026 22:47:04] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [07/Feb/2026 22:47:14] "GET /books HTTP/1.1" 200 -
127.0.0.1 - - [07/Feb/2026 22:48:32] "GET /books HTTP/1.1" 200 -


In [ ]:
import requests

# 1. Define the book data as a clean Python dictionary
new_book = {
    "book_name": "The Great Gatsby",
    "author": "F. Scott Fitzgerald",
    "publisher": "Scribner"
}

# 2. Send the POST request to your running API
response = requests.post("http://127.0.0.1:5000/books", json=new_book)

# 3. Check if it worked
if response.status_code == 201:
    print("Success! Book added to the database.")
    print("Response Data:", response.json())
else:
    print(f"Failed with status code {response.status_code}")
    print("Error details:", response.text)

In [ ]:
# 16.8 Use the sqlalchemy module to connect to the sqlite3 database books.db that you just made in exercise 16.4. As in 16.6,
# select and print the title column from the book table in alphabetical order.

import sqlalchemy

# 1. Connect to the database file you created
engine = sqlalchemy.create_engine('sqlite:///books.db')

# 2. Connect and execute the alphabetical select query
with engine.connect() as connection:
    # We select 'book_name' and use 'ORDER BY' for alphabetical order
    query = sqlalchemy.text("SELECT book_name FROM book ORDER BY book_name ASC")
    result = connection.execute(query)
    
    print("Alphabetical Book Titles:")
    for row in result:
        print(f"- {row[0]}")

In [ ]:
import sqlalchemy

# 1. Connect to the database file (Ensure this matches your Flask config)
engine = sqlalchemy.create_engine('sqlite:///books.db')

# 2. Connect and execute the alphabetical select query
with engine.connect() as connection:
    # Use 'book_name' to match your Flask Model's column name
    query = sqlalchemy.text("SELECT book_name FROM book ORDER BY book_name ASC")
    result = connection.execute(query)
    
    print("Alphabetical Book Titles:")
    print("--------------------------")
    
    # Iterate through the result set and print each title
    for row in result:
        # row[0] accesses the first column (book_name) of the result
        print(f"- {row[0]}")